# XGBoost Cough Detection Training

This notebook reproduces the classical ML pipeline from the research paper for cough detection using multimodal biosignals.

## Objective

Train XGBoost classifiers on three modality configurations:
1. **IMU-only**: 40 handcrafted features from accelerometer and gyroscope
2. **Audio-only**: 65 features from outer microphone (MFCC + spectral + time-domain)
3. **Multimodal**: Combined 105 features (Audio + IMU)

## Expected Results

Based on the paper, 5-fold subject-wise cross-validation should yield:
- IMU-only: ROC-AUC ~0.90 ± 0.02
- Audio-only: ROC-AUC ~0.92 ± 0.01
- Multimodal: ROC-AUC ~0.96 ± 0.01

## Method

- **Window size**: 0.4 seconds (6400 audio samples @ 16kHz, 40 IMU samples @ 100Hz)
- **Data augmentation**: Random temporal shifts (aug_factor=2)
- **Class balancing**: SMOTE oversampling on training splits
- **Feature scaling**: StandardScaler (fit on train, applied to train/val)
- **Cross-validation**: Subject-wise GroupKFold (n=5) to prevent data leakage

In [ ]:
# Check for required dependencies
import sys

try:
    import xgboost
    import imblearn
    print("✓ All required dependencies installed")
    print(f"  - xgboost version: {xgboost.__version__}")
    print(f"  - imbalanced-learn version: {imblearn.__version__}")
except ImportError as e:
    print(f"✗ Missing dependency: {e}")
    print("\nInstall with: pip install xgboost imbalanced-learn shap")
    sys.exit(1)

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats, signal
import librosa
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_auc_score, roc_curve, f1_score, confusion_matrix,
    precision_score, recall_score
)
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

if os.path.exists("/kaggle/usr/lib/"):
    # Load from Kaggle as utility scripts
    from edge_ai_cough_count_helpers import * # pyright: ignore[reportMissingImports]
    from edge_ai_cough_count_dataset_gen import * # pyright: ignore[reportMissingImports]
    from edge_ai_cough_count_features import extract_audio_features, extract_imu_features # pyright: ignore[reportMissingImports]
else:
    # Add src directory to path
    sys.path.append(os.path.abspath('../src'))
    from helpers import *
    from dataset_gen import *
    from features import extract_audio_features, extract_imu_features

print("✓ All imports successful")

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Constants from paper
WINDOW_LEN = 0.4  # 0.4 second windows
AUG_FACTOR = 2    # Data augmentation factor
N_FOLDS = 5       # Number of CV folds

print(f"Configuration:")
print(f"  Window length: {WINDOW_LEN}s")
print(f"  Expected audio samples: {int(WINDOW_LEN * FS_AUDIO)}")
print(f"  Expected IMU samples: {int(WINDOW_LEN * FS_IMU)}")
print(f"  Augmentation factor: {AUG_FACTOR}")
print(f"  CV folds: {N_FOLDS}")

## Data Loading

Load raw windowed data from all 15 subjects using `get_samples_for_subject()` from `dataset_gen.py`.

In [ ]:
# Locate dataset folder
kaggle_dataset_dir = '/kaggle/input/edge-ai-cough-count'
base_dir = kaggle_dataset_dir if os.path.exists(kaggle_dataset_dir) else ".."
data_folder = base_dir + '/public_dataset/'

if not os.path.exists(data_folder):
    raise FileNotFoundError(
        "Cannot find public_dataset/. Please download from: "
        "https://zenodo.org/record/7562332"
    )

# Get list of subject IDs
subject_ids = [d for d in os.listdir(data_folder) 
               if os.path.isdir(os.path.join(data_folder, d))]
subject_ids = sorted(subject_ids)

print(f"✓ Found {len(subject_ids)} subjects: {subject_ids}")

In [ ]:
# Load raw windowed data from all subjects
all_audio = []
all_imu = []
all_labels = []
all_subjects = []

print("Loading dataset (this may take a few minutes)...\n")

for subj_id in tqdm(subject_ids, desc="Loading subjects"):
    try:
        audio, imu, labels, n_coughs = get_samples_for_subject(
            data_folder, subj_id,
            window_len=WINDOW_LEN,
            aug_factor=AUG_FACTOR
        )
        
        all_audio.append(audio)
        all_imu.append(imu)
        all_labels.append(labels)
        all_subjects.extend([subj_id] * len(labels))
        
        print(f"  {subj_id}: {n_coughs} coughs → {len(labels)} windows "
              f"({np.sum(labels==1)} cough, {np.sum(labels==0)} non-cough)")
    except Exception as e:
        print(f"  {subj_id}: Error - {e}")
        continue

# Concatenate all subjects
audio_data = np.concatenate(all_audio, axis=0)
imu_data = np.concatenate(all_imu, axis=0)
labels = np.concatenate(all_labels, axis=0)
subjects = np.array(all_subjects)

print(f"\n{'='*70}")
print(f"Total dataset:")
print(f"  Audio shape: {audio_data.shape}")
print(f"  IMU shape: {imu_data.shape}")
print(f"  Labels: {len(labels)} ({np.sum(labels==1)} coughs, {np.sum(labels==0)} non-coughs)")
print(f"  Unique subjects: {len(np.unique(subjects))}")
print(f"  Class balance: {np.sum(labels==1)/len(labels)*100:.1f}% coughs")
print(f"{'='*70}")

In [ ]:
# Sanity checks
assert audio_data.shape[1] == 6400, f"Expected 6400 audio samples, got {audio_data.shape[1]}"
assert imu_data.shape[1] == 40, f"Expected 40 IMU samples, got {imu_data.shape[1]}"
assert len(np.unique(subjects)) == 15, f"Expected 15 subjects, got {len(np.unique(subjects))}"

# Visualize one cough sample
idx = np.where(labels == 1)[0][0]
fig, axes = plt.subplots(2, 1, figsize=(12, 6))

axes[0].plot(audio_data[idx, :, 0], linewidth=0.5)
axes[0].set_title(f"Sample Cough - Outer Microphone (Subject {subjects[idx]})")
axes[0].set_xlabel("Sample Index")
axes[0].set_ylabel("Amplitude")
axes[0].grid(alpha=0.3)

axes[1].plot(-imu_data[idx, :, 2], linewidth=1)
axes[1].set_title("Accelerometer Z (negated)")
axes[1].set_xlabel("Sample Index")
axes[1].set_ylabel("Acceleration")
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Data loaded and verified successfully")

## Feature Extraction

Extract handcrafted features for all three modalities:
1. IMU-only: 40 features
2. Audio-only: 65 features
3. Multimodal: 105 features

**Note**: This may take 10-20 minutes depending on hardware.

In [ ]:
import multiprocessing
from joblib import Parallel, delayed

print("Extracting features for all modalities...\n")

N = audio_data.shape[0]
n_cpus = multiprocessing.cpu_count()

# Configure parallelization based on available cores
if n_cpus >= 8:
    n_jobs = 8
    n_jobs = 8
    blas_threads = 2
else:
    # Run with all CPUs, but without blas threads
    n_jobs = n_cpus
    blas_threads = 1

# Limit BLAS threading to prevent oversubscription
os.environ['OMP_NUM_THREADS'] = str(blas_threads)
os.environ['OPENBLAS_NUM_THREADS'] = str(blas_threads)
os.environ['MKL_NUM_THREADS'] = str(blas_threads)

print(f"Hardware: {n_cpus} CPU cores detected")
print(f"Configuration: {n_jobs} workers × {blas_threads} BLAS threads = {n_jobs * blas_threads} total\n")

# ===================================================================
# Step 1/2: Extract audio features (65 features from outer mic)
# ===================================================================
print("Step 1/2: Extracting audio features...")
print(f"  Using {n_jobs} parallel workers")

audio_features_list = Parallel(n_jobs=n_jobs, backend='loky')(
    delayed(extract_audio_features)(audio_data[i, :, 0])
    for i in tqdm(range(N), desc="Audio features")
)
X_audio = np.array(audio_features_list)

# Handle NaN/Inf in audio features
if np.any(np.isnan(X_audio)) or np.any(np.isinf(X_audio)):
    print(f"  Warning: Replacing {np.sum(np.isnan(X_audio))} NaN and {np.sum(np.isinf(X_audio))} Inf values in audio")
    X_audio = np.nan_to_num(X_audio, nan=0.0, posinf=0.0, neginf=0.0)

# ===================================================================
# Step 2/2: Extract IMU features (40 features)
# ===================================================================
print("\nStep 2/2: Extracting IMU features...")
print(f"  Using {n_jobs} parallel workers")

imu_features_list = Parallel(n_jobs=n_jobs, backend='loky')(
    delayed(extract_imu_features)(imu_data[i, :, :])
    for i in tqdm(range(N), desc="IMU features")
)
X_imu = np.array(imu_features_list)

# Handle NaN/Inf in IMU features
if np.any(np.isnan(X_imu)) or np.any(np.isinf(X_imu)):
    print(f"  Warning: Replacing {np.sum(np.isnan(X_imu))} NaN and {np.sum(np.isinf(X_imu))} Inf values in IMU")
    X_imu = np.nan_to_num(X_imu, nan=0.0, posinf=0.0, neginf=0.0)

# ===================================================================
# Combine for multimodal (65 audio + 40 IMU = 105 features)
# ===================================================================
X_all = np.concatenate([X_audio, X_imu], axis=1)

print(f"\n{'='*70}")
print(f"Feature extraction complete:")
print(f"  Audio-only: {X_audio.shape} (65 features)")
print(f"  IMU-only: {X_imu.shape} (40 features)")
print(f"  Multimodal: {X_all.shape} (105 features)")
print(f"{'='*70}")

In [ ]:
# Save features to avoid re-extraction
save_path = 'extracted_features.npz'
np.savez(
    save_path,
    X_imu=X_imu, 
    X_audio=X_audio, 
    X_all=X_all,
    labels=labels, 
    subjects=subjects
)
print(f"✓ Features saved to {save_path}")
print(f"  To load: data = np.load('{save_path}')")

## Training Pipeline

Subject-wise cross-validation with:
- GroupKFold (n=5) to prevent data leakage between subjects
- StandardScaler for feature normalization
- SMOTE for handling class imbalance (applied only to training splits)
- XGBoost classifier

In [ ]:
def train_and_evaluate_cv(X, y, groups, n_folds=5, model_name="XGBoost"):
    """
    Subject-wise cross-validation with SMOTE and StandardScaler
    
    Args:
        X: Feature matrix (N, n_features)
        y: Labels (N,)
        groups: Subject IDs (N,)
        n_folds: Number of CV folds
        model_name: Model name for logging
    
    Returns:
        dict: Fold results and metrics
    """
    # Map subject IDs to numeric indices for GroupKFold
    unique_subjects = np.unique(groups)
    subject_to_idx = {subj: idx for idx, subj in enumerate(unique_subjects)}
    group_indices = np.array([subject_to_idx[s] for s in groups])
    
    gkf = GroupKFold(n_splits=n_folds)
    
    results = {
        'fold_aucs': [],
        'fold_predictions': [],
        'fold_true_labels': [],
        'fold_train_subjects': [],
        'fold_val_subjects': []
    }
    
    print(f"\n{'='*70}")
    print(f"Training {model_name} with {n_folds}-fold subject-wise CV")
    print(f"{'='*70}\n")
    
    for fold_idx, (train_idx, val_idx) in enumerate(gkf.split(X, y, group_indices)):
        print(f"Fold {fold_idx + 1}/{n_folds}")
        
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        train_subjects = np.unique(groups[train_idx])
        val_subjects = np.unique(groups[val_idx])
        print(f"  Train: {len(train_subjects)} subjects, {len(y_train)} samples "
              f"({np.sum(y_train==1)} coughs, {np.sum(y_train==0)} non-coughs)")
        print(f"  Val: {len(val_subjects)} subjects, {len(y_val)} samples "
              f"({np.sum(y_val==1)} coughs, {np.sum(y_val==0)} non-coughs)")
        
        # Scale features (fit on train only)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
        
        # Apply SMOTE (train only)
        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)
        print(f"  After SMOTE: {len(y_train_resampled)} samples "
              f"({np.sum(y_train_resampled==1)} coughs, {np.sum(y_train_resampled==0)} non-coughs)")
        
        # Train XGBoost
        model = XGBClassifier(
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            random_state=42,
            eval_metric='logloss',
            verbosity=0
        )
        model.fit(X_train_resampled, y_train_resampled)
        
        # Predict
        y_pred_proba = model.predict_proba(X_val_scaled)[:, 1]
        auc = roc_auc_score(y_val, y_pred_proba)
        print(f"  Validation AUC: {auc:.4f}\n")
        
        results['fold_aucs'].append(auc)
        results['fold_predictions'].append(y_pred_proba)
        results['fold_true_labels'].append(y_val)
        results['fold_train_subjects'].append(train_subjects)
        results['fold_val_subjects'].append(val_subjects)
    
    results['mean_auc'] = np.mean(results['fold_aucs'])
    results['std_auc'] = np.std(results['fold_aucs'])
    
    print(f"\n{'='*70}")
    print(f"CV Results: {results['mean_auc']:.4f} ± {results['std_auc']:.4f}")
    print(f"{'='*70}\n")
    
    return results

print("✓ Training pipeline ready")

In [ ]:
def find_optimal_threshold(results):
    """
    Find threshold that maximizes F1 score across all folds
    
    Args:
        results: Output from train_and_evaluate_cv
    
    Returns:
        best_threshold: Optimal threshold
        best_f1: F1 score at optimal threshold
        thresholds: All tested thresholds
        f1_scores: F1 scores for all thresholds
    """
    all_preds = np.concatenate(results['fold_predictions'])
    all_true = np.concatenate(results['fold_true_labels'])
    
    thresholds = np.linspace(0, 1, 101)
    f1_scores = []
    
    for thresh in thresholds:
        y_pred_binary = (all_preds >= thresh).astype(int)
        f1 = f1_score(all_true, y_pred_binary, zero_division=0)
        f1_scores.append(f1)
    
    best_idx = np.argmax(f1_scores)
    return thresholds[best_idx], f1_scores[best_idx], thresholds, f1_scores

print("✓ Threshold optimization function ready")

In [ ]:
def compute_metrics_at_threshold(results, threshold):
    """
    Compute classification metrics at a specific threshold
    
    Args:
        results: Output from train_and_evaluate_cv
        threshold: Classification threshold
    
    Returns:
        dict: Sensitivity, specificity, precision, F1, confusion matrix
    """
    all_preds = np.concatenate(results['fold_predictions'])
    all_true = np.concatenate(results['fold_true_labels'])
    y_pred_binary = (all_preds >= threshold).astype(int)
    
    tn, fp, fn, tp = confusion_matrix(all_true, y_pred_binary).ravel()
    
    return {
        'threshold': threshold,
        'sensitivity': recall_score(all_true, y_pred_binary),
        'specificity': tn / (tn + fp),
        'precision': precision_score(all_true, y_pred_binary, zero_division=0),
        'f1': f1_score(all_true, y_pred_binary, zero_division=0),
        'tp': int(tp), 'tn': int(tn), 'fp': int(fp), 'fn': int(fn)
    }

print("✓ Metrics computation function ready")

## Experiment 1: IMU-Only Model

Train using only 40 IMU features (accelerometer + gyroscope).

**Expected**: ROC-AUC ~0.90 ± 0.02

In [ ]:
print("="*70)
print("EXPERIMENT 1: IMU-ONLY MODEL")
print("Expected CV AUC: ~0.90 ± 0.02")
print("="*70)

results_imu = train_and_evaluate_cv(
    X_imu, labels, subjects, 
    n_folds=N_FOLDS, 
    model_name="XGBoost (IMU-only)"
)

thresh_imu, f1_imu, _, _ = find_optimal_threshold(results_imu)
metrics_imu = compute_metrics_at_threshold(results_imu, thresh_imu)

print(f"\nOptimal Operating Point:")
print(f"  Threshold: {thresh_imu:.3f}")
print(f"  Sensitivity (Recall): {metrics_imu['sensitivity']:.3f}")
print(f"  Specificity: {metrics_imu['specificity']:.3f}")
print(f"  Precision: {metrics_imu['precision']:.3f}")
print(f"  F1 Score: {metrics_imu['f1']:.3f}")

## Experiment 2: Audio-Only Model

Train using only 65 audio features from the outer microphone.

**Expected**: ROC-AUC ~0.92 ± 0.01

In [ ]:
print("="*70)
print("EXPERIMENT 2: AUDIO-ONLY MODEL (Outer Microphone)")
print("Expected CV AUC: ~0.92 ± 0.01")
print("="*70)

results_audio = train_and_evaluate_cv(
    X_audio, labels, subjects, 
    n_folds=N_FOLDS, 
    model_name="XGBoost (Audio-only)"
)

thresh_audio, f1_audio, _, _ = find_optimal_threshold(results_audio)
metrics_audio = compute_metrics_at_threshold(results_audio, thresh_audio)

print(f"\nOptimal Operating Point:")
print(f"  Threshold: {thresh_audio:.3f}")
print(f"  Sensitivity (Recall): {metrics_audio['sensitivity']:.3f}")
print(f"  Specificity: {metrics_audio['specificity']:.3f}")
print(f"  Precision: {metrics_audio['precision']:.3f}")
print(f"  F1 Score: {metrics_audio['f1']:.3f}")

## Experiment 3: Multimodal Model

Train using all 105 features (65 audio + 40 IMU).

**Expected**: ROC-AUC ~0.96 ± 0.01

In [ ]:
print("="*70)
print("EXPERIMENT 3: MULTIMODAL MODEL (Audio + IMU)")
print("Expected CV AUC: ~0.96 ± 0.01")
print("="*70)

results_all = train_and_evaluate_cv(
    X_all, labels, subjects, 
    n_folds=N_FOLDS, 
    model_name="XGBoost (Multimodal)"
)

thresh_all, f1_all, _, _ = find_optimal_threshold(results_all)
metrics_all = compute_metrics_at_threshold(results_all, thresh_all)

print(f"\nOptimal Operating Point:")
print(f"  Threshold: {thresh_all:.3f}")
print(f"  Sensitivity (Recall): {metrics_all['sensitivity']:.3f}")
print(f"  Specificity: {metrics_all['specificity']:.3f}")
print(f"  Precision: {metrics_all['precision']:.3f}")
print(f"  F1 Score: {metrics_all['f1']:.3f}")

## Results Summary

Comparison of all three modalities:

In [ ]:
# Create summary table
summary_df = pd.DataFrame({
    'Model': ['IMU-only', 'Audio-only', 'Multimodal'],
    'ROC-AUC': [
        f"{results_imu['mean_auc']:.4f} ± {results_imu['std_auc']:.4f}",
        f"{results_audio['mean_auc']:.4f} ± {results_audio['std_auc']:.4f}",
        f"{results_all['mean_auc']:.4f} ± {results_all['std_auc']:.4f}"
    ],
    'Sensitivity': [
        f"{metrics_imu['sensitivity']:.3f}",
        f"{metrics_audio['sensitivity']:.3f}",
        f"{metrics_all['sensitivity']:.3f}"
    ],
    'Specificity': [
        f"{metrics_imu['specificity']:.3f}",
        f"{metrics_audio['specificity']:.3f}",
        f"{metrics_all['specificity']:.3f}"
    ],
    'Precision': [
        f"{metrics_imu['precision']:.3f}",
        f"{metrics_audio['precision']:.3f}",
        f"{metrics_all['precision']:.3f}"
    ],
    'F1': [
        f"{metrics_imu['f1']:.3f}",
        f"{metrics_audio['f1']:.3f}",
        f"{metrics_all['f1']:.3f}"
    ]
})

print("\n" + "="*80)
print("FINAL RESULTS SUMMARY")
print("="*80)
print(summary_df.to_string(index=False))
print("\n" + "="*80)
print("Expected from paper:")
print("  IMU-only:    0.90 ± 0.02")
print("  Audio-only:  0.92 ± 0.01")
print("  Multimodal:  0.96 ± 0.01")
print("="*80)

## Visualization 1: ROC Curves

Plot ROC curves for all folds of each modality:

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (results, name, color) in enumerate([
    (results_imu, 'IMU-only', 'blue'),
    (results_audio, 'Audio-only', 'green'),
    (results_all, 'Multimodal', 'red')
]):
    ax = axes[idx]
    
    # Plot each fold
    for fold_idx in range(N_FOLDS):
        y_true = results['fold_true_labels'][fold_idx]
        y_pred = results['fold_predictions'][fold_idx]
        fpr, tpr, _ = roc_curve(y_true, y_pred)
        auc = results['fold_aucs'][fold_idx]
        ax.plot(fpr, tpr, alpha=0.3, color=color, 
                label=f'Fold {fold_idx+1} (AUC={auc:.3f})')
    
    ax.plot([0, 1], [0, 1], 'k--', label='Random', linewidth=2)
    ax.set_xlabel('False Positive Rate', fontsize=12)
    ax.set_ylabel('True Positive Rate', fontsize=12)
    ax.set_title(f'{name}\nMean AUC: {results["mean_auc"]:.4f} ± {results["std_auc"]:.4f}',
                fontsize=13, fontweight='bold')
    ax.legend(fontsize=9, loc='lower right')
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('roc_curves_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ ROC curves saved to roc_curves_comparison.png")

## Visualization 2: Confusion Matrices

Show classification results at optimal thresholds:

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, (metrics, name) in enumerate([
    (metrics_imu, 'IMU-only'),
    (metrics_audio, 'Audio-only'),
    (metrics_all, 'Multimodal')
]):
    ax = axes[idx]
    cm = np.array([[metrics['tn'], metrics['fp']], 
                   [metrics['fn'], metrics['tp']]])
    
    im = ax.imshow(cm, cmap='Blues', interpolation='nearest')
    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(['Non-cough', 'Cough'])
    ax.set_yticklabels(['Non-cough', 'Cough'])
    ax.set_xlabel('Predicted', fontsize=11)
    ax.set_ylabel('True', fontsize=11)
    ax.set_title(f'{name}\nF1={metrics["f1"]:.3f} (thresh={metrics["threshold"]:.2f})',
                fontsize=12, fontweight='bold')
    
    # Add text annotations
    for i in range(2):
        for j in range(2):
            ax.text(j, i, cm[i, j], ha='center', va='center',
                   color='white' if cm[i, j] > cm.max()/2 else 'black',
                   fontsize=16, fontweight='bold')
    
    plt.colorbar(im, ax=ax, fraction=0.046)

plt.tight_layout()
plt.savefig('confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Confusion matrices saved to confusion_matrices.png")

## Visualization 3: F1 Score vs Threshold

Show how F1 score varies with classification threshold:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

for results, name, color, metrics in [
    (results_imu, 'IMU-only', 'blue', metrics_imu),
    (results_audio, 'Audio-only', 'green', metrics_audio),
    (results_all, 'Multimodal', 'red', metrics_all)
]:
    thresh, best_f1, thresholds, f1_scores = find_optimal_threshold(results)
    ax.plot(thresholds, f1_scores, 
            label=f'{name} (max F1={best_f1:.3f} @ {thresh:.2f})',
            color=color, linewidth=2)
    ax.axvline(thresh, color=color, linestyle='--', alpha=0.5, linewidth=1)

ax.set_xlabel('Classification Threshold', fontsize=12)
ax.set_ylabel('F1 Score', fontsize=12)
ax.set_title('F1 Score vs Classification Threshold', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('f1_vs_threshold.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ F1 vs threshold plot saved to f1_vs_threshold.png")

## Visualization 4: Per-Fold AUC Comparison

Compare AUC scores across all folds for each modality:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(N_FOLDS)
width = 0.25

ax.bar(x - width, results_imu['fold_aucs'], width, 
       label='IMU-only', color='blue', alpha=0.7)
ax.bar(x, results_audio['fold_aucs'], width, 
       label='Audio-only', color='green', alpha=0.7)
ax.bar(x + width, results_all['fold_aucs'], width, 
       label='Multimodal', color='red', alpha=0.7)

# Add mean lines
ax.axhline(results_imu['mean_auc'], color='blue', linestyle='--', 
          alpha=0.5, linewidth=2, label=f'IMU mean: {results_imu["mean_auc"]:.3f}')
ax.axhline(results_audio['mean_auc'], color='green', linestyle='--', 
          alpha=0.5, linewidth=2, label=f'Audio mean: {results_audio["mean_auc"]:.3f}')
ax.axhline(results_all['mean_auc'], color='red', linestyle='--', 
          alpha=0.5, linewidth=2, label=f'Multimodal mean: {results_all["mean_auc"]:.3f}')

ax.set_xlabel('Fold', fontsize=12)
ax.set_ylabel('ROC-AUC', fontsize=12)
ax.set_title('Per-Fold AUC Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([f'Fold {i+1}' for i in range(N_FOLDS)])
ax.legend(fontsize=10)
ax.grid(alpha=0.3, axis='y')
ax.set_ylim(0.8, 1.0)
plt.tight_layout()
plt.savefig('per_fold_auc.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Per-fold AUC comparison saved to per_fold_auc.png")

## Summary

Successfully reproduced the paper's XGBoost training pipeline with three modality configurations.

### Key Findings

1. **Multimodal fusion** (audio + IMU) achieves best performance (~0.96 AUC)
2. **Audio alone** is strong (~0.92 AUC) - outer microphone captures cough signatures well
3. **IMU adds value** - provides ~4% AUC improvement when combined with audio
4. **Subject-wise CV** ensures generalization to new subjects
5. **Class balancing** with SMOTE improves performance on imbalanced data

### Model Comparison

- **IMU-only**: Good baseline using motion sensors alone (useful for privacy-preserving scenarios)
- **Audio-only**: Strong performance, but may struggle in noisy environments
- **Multimodal**: Best of both worlds - robust across conditions

### Next Steps

1. **Feature selection**: Use RFECV to reduce feature count while maintaining performance
2. **Hyperparameter tuning**: RandomizedSearchCV or Optuna for optimal XGBoost parameters
3. **Explainability**: SHAP analysis to understand which features drive predictions
4. **Final validation**: Test on held-out subjects for unbiased performance estimate
5. **Edge deployment**: Model quantization and optimization for resource-constrained devices
6. **Real-time inference**: Implement sliding window approach for continuous monitoring

### Files Generated

- `extracted_features.npz`: Cached features (can be reloaded to skip extraction)
- `roc_curves_comparison.png`: ROC curves for all modalities
- `confusion_matrices.png`: Classification results at optimal thresholds
- `f1_vs_threshold.png`: F1 score sensitivity to threshold choice
- `per_fold_auc.png`: Cross-validation stability analysis

## Model Persistence for Interactive Testing

Save trained models to disk for use in the Interactive Model Testing notebook.

This trains final models on **all available data** (with SMOTE balancing) and saves:
- Trained XGBoost model
- StandardScaler fitted on full dataset
- Optimal classification threshold (from CV)

In [ ]:
# Save trained models for interactive testing
import pickle
from pathlib import Path

MODEL_DIR = Path("./models")
MODEL_DIR.mkdir(exist_ok=True)

# Store models in memory for SHAP analysis
trained_models = {}

print("Training final models on all data for deployment...\n")

for modality, X, thresh in [
    ('imu', X_imu, thresh_imu),
    ('audio', X_audio, thresh_audio),
    ('multimodal', X_all, thresh_all)
]:
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Apply SMOTE for balanced training
    smote = SMOTE(random_state=42)
    X_balanced, y_balanced = smote.fit_resample(X_scaled, labels)
    
    # Train final model
    model = XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        eval_metric='logloss',
        verbosity=0
    )
    model.fit(X_balanced, y_balanced)
    
    # Store in memory for SHAP
    trained_models[modality] = {
        'model': model,
        'scaler': scaler,
        'X_balanced': X_balanced,
        'y_balanced': y_balanced,
        'threshold': thresh
    }
    
    # Save model + scaler + threshold
    model_data = {
        'model': model,
        'scaler': scaler,
        'threshold': thresh
    }
    
    save_path = MODEL_DIR / f'xgb_{modality}.pkl'
    with open(save_path, 'wb') as f:
        pickle.dump(model_data, f)
    
    print(f"✓ Saved {modality} model to {save_path}")

print(f"\n✓ All models saved to {MODEL_DIR}/")
print("  These can now be loaded in Interactive_Model_Testing.ipynb")

## SHAP Explainability Analysis

Understand which features drive cough detection predictions using SHAP (SHapley Additive exPlanations).

SHAP provides:
- **Global explanations**: Which features are most important overall
- **Local explanations**: How each feature contributes to individual predictions
- **Feature interactions**: How features work together

We use the final models trained in the previous section (on full dataset with SMOTE balancing) to compute SHAP values.

In [ ]:
# Check SHAP installation
try:
    import shap
    print(f"✓ SHAP version: {shap.__version__}")
except ImportError:
    print("✗ SHAP not installed. Install with: pip install shap")
    import sys
    sys.exit(1)

# Import feature name functions from features.py
from features import get_feature_names

# Create output directory for SHAP plots
SHAP_OUTPUT_DIR = Path("./shap")
SHAP_OUTPUT_DIR.mkdir(exist_ok=True)
print(f"✓ SHAP output directory: {SHAP_OUTPUT_DIR}")

# Verify feature name generation
print(f"\nFeature name counts:")
print(f"  Audio: {len(get_feature_names('audio'))}")
print(f"  IMU: {len(get_feature_names('imu'))}")
print(f"  Multimodal: {len(get_feature_names('multimodal'))}")

In [ ]:
# Compute SHAP values using the trained models from cell 33
shap_results = {}

for modality, modality_name in [
    ('imu', 'IMU-only'),
    ('audio', 'Audio-only'),
    ('multimodal', 'Multimodal')
]:
    print(f"\n{'='*70}")
    print(f"Computing SHAP values: {modality_name}")
    print(f"{'='*70}\n")

    # Retrieve stored model and data
    stored = trained_models[modality]
    model = stored['model']
    X_balanced = stored['X_balanced']
    y_balanced = stored['y_balanced']

    # Get feature names from features.py
    feature_names = get_feature_names(modality)

    print(f"Dataset: {len(X_balanced)} samples (SMOTE-balanced)")
    print(f"Features: {X_balanced.shape[1]} ({modality_name})")
    print(f"Computing SHAP values using TreeExplainer...")

    # Compute SHAP values
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_balanced)

    print(f"✓ SHAP values computed")

    # Store for visualization
    shap_results[modality] = {
        'shap_values': shap_values,
        'model': model,
        'X_balanced': X_balanced,
        'y_balanced': y_balanced,
        'feature_names': feature_names
    }

print(f"\n{'='*70}")
print("SHAP computation complete for all modalities")
print(f"{'='*70}\n")

In [ ]:
# Generate SHAP visualizations for each modality

for modality_key, modality_name in [
    ('imu', 'IMU-only'),
    ('audio', 'Audio-only'),
    ('multimodal', 'Multimodal')
]:
    print(f"\nGenerating SHAP plots for {modality_name}...")

    result = shap_results[modality_key]
    shap_values = result['shap_values']
    X_balanced = result['X_balanced']
    feature_names = result['feature_names']

    # =====================================================================
    # Visualization 1: Summary Bar Plot (Top 20 features)
    # =====================================================================
    plt.figure(figsize=(10, 8))
    shap.summary_plot(
        shap_values,
        X_balanced,
        feature_names=feature_names,
        plot_type="bar",
        max_display=20,
        show=False
    )
    plt.title(f'SHAP Feature Importance: {modality_name}',
              fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    save_path = SHAP_OUTPUT_DIR / f'summary_bar_{modality_key}.png'
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"  ✓ Saved {save_path}")

    # =====================================================================
    # Visualization 2: Beeswarm Plot (Top 20 features)
    # =====================================================================
    plt.figure(figsize=(10, 8))
    shap.summary_plot(
        shap_values,
        X_balanced,
        feature_names=feature_names,
        max_display=20,
        show=False
    )
    plt.title(f'SHAP Value Distribution: {modality_name}',
              fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    save_path = SHAP_OUTPUT_DIR / f'summary_beeswarm_{modality_key}.png'
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"  ✓ Saved {save_path}")

    # =====================================================================
    # Visualization 3: Dependence Plots (Top 3 features)
    # =====================================================================
    # Compute feature importance to identify top features
    feature_importance = np.abs(shap_values).mean(axis=0)
    top_indices = np.argsort(feature_importance)[-3:][::-1]  # Top 3

    for rank, idx in enumerate(top_indices, 1):
        feature_name = feature_names[idx]

        plt.figure(figsize=(10, 6))
        shap.dependence_plot(
            idx,
            shap_values,
            X_balanced,
            feature_names=feature_names,
            show=False
        )
        plt.title(f'SHAP Dependence: {feature_name} ({modality_name})',
                  fontsize=14, fontweight='bold')
        plt.tight_layout()
        save_path = SHAP_OUTPUT_DIR / f'dependence_{rank}_{modality_key}.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"  ✓ Saved {save_path} (feature: {feature_name})")

print(f"\n{'='*70}")
print(f"All SHAP visualizations saved to {SHAP_OUTPUT_DIR}/")
print(f"{'='*70}\n")

# Display summary of top features per modality
print("\nTop 5 Most Important Features per Modality:\n")

for modality_key, modality_name in [
    ('imu', 'IMU-only'),
    ('audio', 'Audio-only'),
    ('multimodal', 'Multimodal')
]:
    result = shap_results[modality_key]
    shap_values = result['shap_values']
    feature_names = result['feature_names']

    feature_importance = np.abs(shap_values).mean(axis=0)
    top_5_indices = np.argsort(feature_importance)[-5:][::-1]

    print(f"{modality_name}:")
    for rank, idx in enumerate(top_5_indices, 1):
        print(f"  {rank}. {feature_names[idx]}: {feature_importance[idx]:.4f}")
    print()

print("✓ SHAP analysis complete")